In [1]:
# Delete this cell to re-enable tracebacks
import sys
ipython = get_ipython()

def hide_traceback(exc_tuple=None, filename=None, tb_offset=None,
                   exception_only=False, running_compiled_code=False):
    etype, value, tb = sys.exc_info()
    return ipython._showtraceback(etype, value, ipython.InteractiveTB.get_exception_only(etype, value))

ipython.showtraceback = hide_traceback

In [2]:
# JSON output syntax highlighting
from __future__ import print_function
from pygments import highlight
from pygments.lexers import JsonLexer
from pygments.formatters import HtmlFormatter
from IPython.display import HTML

original_print = print

def json_print(inpt):
    string = str(inpt)
    if string[0] == '{':
        formatter = HtmlFormatter()
        return HTML('<style type="text/css">{}</style>{}'.format(
                    formatter.get_style_defs('.highlight'),
                    highlight(string, JsonLexer(), formatter)))
    else:
        original_print(inpt)

print = json_print

## TAXIICollection

The TAXIICollection suite contains **TAXIICollectionStore**, **TAXIICollectionSource**, and **TAXIICollectionSink**. **TAXIICollectionStore** for pushing and retrieving STIX content to local/remote TAXII Collection(s). **TAXIICollectionSource** for retrieving STIX content to local/remote TAXII Collection(s). **TAXIICollectionSink** for pushing STIX content to local/remote TAXII Collection(s). Each of the interfaces is designed to be binded to a Collection from the taxii2client library (taxii2client.Collection), where all **TAXIICollection** API calls will be executed through that Collection instance.

A note on TAXII2 searching/filtering of STIX content. TAXII2 server implementations natively support searching on the STIX2 object properties: id, type and version; API requests made to TAXII2 can contain filter arguments for those 3 properties. However, the **TAXIICollection** suite supports searching on all STIX2 common object properties (see **Filters** documentation for full listing). This works simply by augmenting the filtering that is done remotely at the TAXII2 server instance. **TAXIICollection** will seperate any supplied queries into TAXII supported filters and non-supported filters. During a **TAXIICollection** API call, TAXII2 supported filters get inserted into the TAXII2 server request (to be evaluated at the server). The rest of the filters are kept locally and then applied to the STIX2 content that is returned from the TAXII2 server, before being returned from the **TAXIICollection** API call. 

### TAXIICollection API

### TAXIICollection Examples

#### TAXIICollectionSource

In [1]:
from stix2 import TAXIICollectionSource
from taxii2client import Collection

# establish TAXII2 Collection instance
collection = Collection("https://test.freetaxii.com:8000/api1/collections/9cfa669c-ee94-4ece-afd2-f8edac37d8fd/")
# supply the TAXII2 collection to TAXIICollection
tc_source = TAXIICollectionSource(collection)

#retrieve STIX object by id
stix_obj = tc_source.get("indicator--0f63229c-07a2-46dd-939d-312c7bf6d114")

#for visual purposes
print(stix_obj)


{
    "type": "indicator",
    "id": "indicator--fb2c0e55-52a0-423c-b544-8b09622cafc1",
    "created": "2017-10-02T19:26:30.000Z",
    "modified": "2017-10-02T19:26:30.000Z",
    "labels": [
        "malicious-activity"
    ],
    "name": "Emerging Threats - Block Rules - Compromised IPs",
    "pattern": "[ ipv4-addr:value = '98.138.19.88' ]",
    "valid_from": "2017-10-02T19:26:30Z",
    "kill_chain_phases": [
        {
            "kill_chain_name": "lockheed-martin-cyber-kill-chain",
            "phase_name": "delivery"
        }
    ]
}


In [11]:
from stix2 import Filter

# retrieve multiple object from TAXIICollectionSource
# by using filters
f1 = Filter("type","=", "indicator")

indicators = tc_source.query([f1])

#for visual purposes
print("indicators: {0}").format(str(len(indicators)))
for indicator in indicators:
    print(indicator)

indicators: 126
{
    "type": "indicator",
    "id": "indicator--569b8969-bfce-4ab4-9a45-06ce78799a35",
    "created": "2017-10-02T20:40:44.000Z",
    "modified": "2017-10-02T20:40:44.000Z",
    "labels": [
        "malicious-activity"
    ],
    "name": "Emerging Threats - Block Rules - Compromised IPs",
    "pattern": "[ ipv4-addr:value = '207.158.1.150' ]",
    "valid_from": "2017-10-02T20:40:44Z",
    "kill_chain_phases": [
        {
            "kill_chain_name": "lockheed-martin-cyber-kill-chain",
            "phase_name": "delivery"
        }
    ]
}
{
    "type": "indicator",
    "id": "indicator--9c418633-9970-424e-8030-2c3dfa3105da",
    "created": "2017-10-02T20:40:44.000Z",
    "modified": "2017-10-02T20:40:44.000Z",
    "labels": [
        "malicious-activity"
    ],
    "name": "Emerging Threats - Block Rules - Compromised IPs",
    "pattern": "[ ipv4-addr:value = '64.4.30.34' ]",
    "valid_from": "2017-10-02T20:40:44Z",
    "kill_chain_phases": [
        {
            "

#### TAXIICollectionSink

In [ ]:
from stix2 import TAXIICollectionSink, ThreatActor

#create TAXIICollectionSINK and push STIX content to it
tc_sink = TAXIICollectionSink(collection)

# create new STIX threat-actor
ta = ThreatActor(name="Teddy Bear",
                labels=["nation-state"],
                sophistication="innovator",
                resource_level="government",
                goals=[
                    "compromising environment NGOs",
                    "water-hole attacks geared towards energy sector",
                ])

tc_sink.add(ta)




#### TAXIICollectionStore

In [8]:
from stix2 import TAXIICollectionStore

# create TAXIICollectionStore - note the same collection instance can
# be used for the store
tc_store = TAXIICollectionStore(collection)

# retrieve STIX object by id from TAXII Collection through
# TAXIICollectionStore
stix_obj2 = tc_source.get("indicator--6850d393-36b6-4a67-ad45-f9e4d512c799")

print(stix_obj2)

{
    "type": "indicator",
    "id": "indicator--d8e1cd37-4a6c-4088-aded-ed79c4ea2caa",
    "created": "2017-10-02T20:24:03.000Z",
    "modified": "2017-10-02T20:24:03.000Z",
    "labels": [
        "malicious-activity"
    ],
    "name": "Emerging Threats - Block Rules - Compromised IPs",
    "pattern": "[ ipv4-addr:value = '98.138.19.88' ]",
    "valid_from": "2017-10-02T20:24:03Z",
    "kill_chain_phases": [
        {
            "kill_chain_name": "lockheed-martin-cyber-kill-chain",
            "phase_name": "delivery"
        }
    ]
}


In [ ]:
from stix2 import indicator

# add STIX object to TAXIICollectionStore
ind = Indicator(description="Smokey Bear implant",
                labels=["malicious-activity"],
                pattern="[file:hashes.'SHA-256' = '09c7e05a39a59428743635242e4a867c932140a909f12a1e54fa7ee6a440c73b']")

tc_store.add(ind)
